# <center>Segmenting and Clustering Neighborhoods in Toronto</center>

## Getting Data

We will get data from wikipedia website using Web Scraping

In [85]:
#import libraries
from bs4 import BeautifulSoup #library for web scraping
import requests

In [86]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [87]:
table = soup.table
table

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:80%;"><a hr

In [210]:
for cell in table.find_all('td'):
    try:
        postcode = cell.b.text
        postcode
        borough = cell.span.a.text
        neigh = cell.span.text
        neighbor = ''.join(neigh.split('(')[1].split(')'))
        neighborhood = neighbor.replace('/',',')
    except Exception as e:
        borough = 'Not Assigned'
        neighborhood = 'Not Assigned'
    
    #split neighborhood by brackets and repace / with ,
    print("Postal Code : {} \nBorough : {} \nNeighborhood : {}\n\n".format(postcode,borough,neighborhood))

Postal Code : M1A 
Borough : Not Assigned 
Neighborhood : Not Assigned


Postal Code : M2A 
Borough : Not Assigned 
Neighborhood : Not Assigned


Postal Code : M3A 
Borough : North York 
Neighborhood : Parkwoods


Postal Code : M4A 
Borough : North York 
Neighborhood : Victoria Village


Postal Code : M5A 
Borough : Downtown Toronto 
Neighborhood : Regent Park , Harbourfront


Postal Code : M6A 
Borough : North York 
Neighborhood : Lawrence Manor , Lawrence Heights


Postal Code : M7A 
Borough : Not Assigned 
Neighborhood : Not Assigned


Postal Code : M8A 
Borough : Not Assigned 
Neighborhood : Not Assigned


Postal Code : M9A 
Borough : Etobicoke 
Neighborhood : Islington Avenue


Postal Code : M1B 
Borough : Scarborough 
Neighborhood : Malvern , Rouge


Postal Code : M2B 
Borough : Not Assigned 
Neighborhood : Not Assigned


Postal Code : M3B 
Borough : North York 
Neighborhood : Don MillsNorth


Postal Code : M4B 
Borough : East York 
Neighborhood : Parkview Hill , Woodbine Gardens

In [195]:
cell = table.find_all('td')
cell[17]

<td style="vertical-align:top;">
<p><b>M9B</b><br/><span style="font-size:80%;"><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a><br/>(<a class="mw-redirect" href="/wiki/West_Deane_Park" title="West Deane Park">West Deane Park</a> / <a href="/wiki/Princess_Gardens" title="Princess Gardens">Princess Gardens</a> / <a class="mw-redirect" href="/wiki/Martin_Grove_Road" title="Martin Grove Road">Martin Grove</a> / <a class="mw-redirect" href="/wiki/Islington,_Toronto" title="Islington, Toronto">Islington</a> / Cloverdale)</span>
</p>
</td>

In [197]:
postcode = cell.b.text
postcode

AttributeError: ResultSet object has no attribute 'b'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [198]:
try:
    borough = cell.span.a.text
    borough
except Exception as e:
    borough = 'Not Assigned'
borough

'Not Assigned'

In [199]:
try:
    neigh = cell.span.text
    neigh
except Exception as e:
    neigh = "Not Assigned"
neigh

'Not Assigned'

In [200]:
#split neighborhood by brackets and repace / with ,
try:
    neighbor = ''.join(neigh.split('(')[1].split(')'))
    neighborhood = neighbor.replace('/',',')
except Exception as e:
    neighborhood = "Not Assigned"
neighborhood

'Not Assigned'

In [201]:
neighborhood

'Not Assigned'

In [202]:
postcode,borough,neighborhood

('M9B', 'Not Assigned', 'Not Assigned')

In [211]:
import pandas as pd
data = [{'postcode':postcode,'borough':borough,'neighbourhood':neighborhood}]
pd.DataFrame(data)

,borough,neighbourhood,postcode
0,Not Assigned,Not Assigned,M9Z
